In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('../../main_dataset/data_all.csv')

weather_df = pd.read_csv('../../main_dataset/context_weather_all_temp_discr.csv', parse_dates=['data'])
weather_df.rename({'data' :'date'}, inplace=True, axis=1)
weather_df.set_index('date', inplace=True)

pop_df = pd.read_csv('../../main_dataset/context_poi_popularity.csv', index_col='Unnamed: 0')
pop_df.rename(columns={'temp_discr' :'temp'}, inplace=True)

/tmp/ipykernel_57726/2426495986.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../main_dataset/data_all.csv')


In [11]:
df

,id_veronacard,profilo,data_visita,ora_visita,sito_nome,poi
0,04AA72F2185080,vrcard-24-2019,2019-08-22,17:03:00,Torre dei Lamberti,59
1,0419C5E2185081,vrcard-48-2019,2019-12-01,11:46:00,Torre dei Lamberti,59
2,042C0FE2185085,vrcard-48-2019,2019-07-31,13:37:00,Torre dei Lamberti,59
3,042B2DE2185085,vrcard-48-2019,2019-07-31,13:37:00,Torre dei Lamberti,59
4,04191AE2185081,vrcard-48-2019,2019-12-01,11:46:00,Torre dei Lamberti,59
...,...,...,...,...,...,...
4448589,0440ADE2185081,vrcard-48-2019,2020-01-01,10:56:07,Giardino Giusti,75
4448590,04161EE2185081,vrcard-48-2019,2020-01-01,10:55:40,Giardino Giusti,75
4448591,044452E2185081,vrcard-48-2019,2020-01-01,10:55:27,Giardino Giusti,75
4448592,043412E2185085,vrcard-48-2019,2020-01-01,10:22:41,Giardino Giusti,75


In [12]:
df['date'] = df['data_visita'] + ' ' + df['ora_visita']
df = df[['sito_nome', 'poi', 'date']]
df['date'] = pd.to_datetime(df.loc[:, 'date'])

/tmp/ipykernel_57726/3075646354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df.loc[:, 'date'])


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448594 entries, 0 to 4448593
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   sito_nome  object        
 1   poi        int64         
 2   date       datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 101.8+ MB


In [14]:
df['sito_nome'].unique()

array(['Torre dei Lamberti', 'Anfiteatro Arena', 'Casa di Giulietta',
       'Museo di Castelvecchio', 'complesso del Duomo', 'Giardino Giusti',
       'Museo Africano', 'Museo Lapidario Maffeiano',
       'Museo di Storia Naturale', 'Palazzo della Ragione o del Comune',
       'Chiesa di San Fermo', 'Basilica di San Zeno',
       'Basilica di Santa Anastasia',
       'Museo archeologico al teatro romano',
       'Museo degli Affreschi "G.B. Cavalcaselle" e Tomba di Giulietta',
       'Teatro Romano', 'Castelvecchio'], dtype=object)

In [37]:
df_dic = {}
df_count_dic = {}
for poi in df['sito_nome'].unique():
    mask = df['sito_nome'] == poi
    df_dic[poi] = df.loc[mask]
    #df_dic[poi].reset_index(drop=True, inplace=True)
    df_dic[poi].set_index('date', inplace=True)
    df_count_dic[poi] = df_dic[poi].resample('15min').size().reset_index(name='count')
    
    new_index = pd.date_range(
    start=weather_df.index.min(),
    end=weather_df.index.max() + pd.Timedelta(days=1) - pd.Timedelta(minutes=15),
    freq='15min', name='date'
    )
    df_meteo = pd.DataFrame(index=new_index)
    df_meteo = df_meteo.merge(weather_df, left_index=True, right_index=True, how='left')
    df_meteo.ffill(inplace=True)
    df_count_dic[poi] = df_count_dic[poi].merge(df_meteo, on='date', how='left')
    
    df_popularity = pop_df[pop_df['poi_name'] == poi]
    df_count_dic[poi] = df_count_dic[poi].merge(df_popularity, on=['rain', 'temp'], how='left').drop(['poi_name', 'poi'], axis=1)
    
    df_count_dic[poi].replace({'no_rain': 0, 'rain': 1}, inplace=True)
    mask_2020 = df_count_dic[poi].date < pd.to_datetime('2020-01-01')
    df_count_dic[poi] = df_count_dic[poi][mask_2020]

/tmp/ipykernel_57726/3324847670.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_count_dic[poi].replace({'no_rain': 0, 'rain': 1}, inplace=True)
/tmp/ipykernel_57726/3324847670.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_count_dic[poi].replace({'no_rain': 0, 'rain': 1}, inplace=True)
/tmp/ipykernel_57726/3324847670.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in

In [40]:
df_count_dic['Casa di Giulietta'].tail()

,date,count,temp,rain,popularity
210277,2019-12-31 22:45:00,0,0.0,0,17.488233
210278,2019-12-31 23:00:00,0,0.0,0,17.488233
210279,2019-12-31 23:15:00,0,0.0,0,17.488233
210280,2019-12-31 23:30:00,0,0.0,0,17.488233
210281,2019-12-31 23:45:00,0,0.0,0,17.488233


In [44]:
for poi in df_count_dic.keys():
    if len(df_count_dic[poi]) == 0:
        pass
    else:
    	df_string = poi.lower().replace(' ', '_')
    	df_count_dic[poi].to_csv(f'../../main_dataset/count_data_2020/data_{df_string}.csv')